# Import Library

In [1]:
import os
import re
import json
import xml.etree.ElementTree as ET
from pathlib import Path

import pandas as pd
import numpy as np

# Functions

In [2]:
def get_namespace(element):
    m = re.match(r'\{.*\}', element.tag)
    if m:
        tag = m.group(0)
    else:
        tag = ''
    return  tag

In [3]:
def parse_data(root):
    list_records = list()
    xml_nms = get_namespace(root)

    list_elems = root.findall(f'{xml_nms}logitem')
    n_all_elems = len(list_elems)
    nth_percent_display = n_all_elems // 10

    xml_tag_mapper = {
        f'{xml_nms}id': 'log_id',
        f'{xml_nms}timestamp': 'log_timestamp',
        f'{xml_nms}contributor/{xml_nms}username': 'user_name',
        f'{xml_nms}contributor/{xml_nms}id': 'user_id',
        f'{xml_nms}comment': 'user_comment',
        f'{xml_nms}type': 'action_type',
        f'{xml_nms}action': 'action',
        f'{xml_nms}logtitle': 'log_title'
    }

    for i, elem in enumerate(list_elems):
        record = dict()

        for xml_tag, col_name in xml_tag_mapper.items():
            try:
                value = elem.find(xml_tag).text
            except AttributeError:
                value = None
            
            record[col_name] = value

        list_records.append(record)

        if i % nth_percent_display == 0:
            progress = int(i / nth_percent_display)
            if progress <= 9 and progress > 0:
                print("progress: {}% {}".format(10 * progress, "." * progress))
            else:
                if i == 10:
                    print("done")

    df = pd.read_json(json.dumps(list_records), orient='records')

    return df

In [4]:
def parse_column_header(sql):
    s = re.search(r'CREATE TABLE .* \(((.* NOT NULL,)+)', sql, re.DOTALL)
    if s:
        col_names = s.group(1)
        col_names = col_names.replace('\n', '')
        col_names = ' '.join(col_names.split())
        
        list_new_col_names = list()

        for col_name in col_names.split(','):
            if len(col_name) > 0:
                m = re.match(r'(.*) .*\(\d+\).* NOT NULL', col_name)
                col_name = m.group(1)
                col_name = col_name.replace('`', '').strip()
                list_new_col_names.append(col_name)
        
        return list_new_col_names
    else:
        return None

In [5]:
def parse_query(sql):
    lang_proficiency = re.search('INSERT INTO .* VALUES (.*);', sql).group(1)
    list_lang_prof = lang_proficiency.replace('),(', ')_(').split('_')
    list_lang_prof = list(map(lambda x: x[1:-1].replace('\'', '').split(','), list_lang_prof))
    
    header = parse_column_header(sql)
    if header is None:
        header = ['babel_user', 'babel_lang', 'babel_level']

    df = pd.DataFrame(list_lang_prof, columns=header)

    return df

In [6]:
def process_log(df):
    df['log_timestamp'] = df['log_timestamp'].str.replace('T', ' ')
    df['log_timestamp'] = df['log_timestamp'].str.replace('Z', '')
    df['log_timestamp'] = pd.to_datetime(df['log_timestamp'], format='%Y-%m-%d %H:%M:%S')

    df.drop(columns=['user_name', 'log_title'], inplace=True)

    df.dropna(subset='user_id', inplace=True)
    df['user_id'] = df['user_id'].astype(int)

    df.sort_values(by=['user_id', 'log_timestamp'], ascending=[True, True], inplace=True, ignore_index=True)

In [7]:
def get_log_unblock(df):
    df_user_block = df.loc[df['action'].isin(['block', 'reblock']), ['log_id', 'user_id', 'log_timestamp']]
    df_user_unblock = df.loc[df['action'].eq('unblock'), ['user_id', 'log_timestamp', 'user_comment']]

    df_user_block.sort_values(by='log_timestamp', inplace=True)
    df_user_unblock.sort_values(by='log_timestamp', inplace=True)

    df_comment = pd.merge_asof(df_user_block, df_user_unblock, by='user_id', on='log_timestamp', tolerance=pd.Timedelta('2d'), direction='forward')
    df_comment['user_comment'] = df_comment['user_comment'].fillna('')
    df_comment['user_comment'] = df_comment['user_comment'].str.lower()

    list_log_id = df_comment.loc[df_comment['user_comment'].str.contains('ผิด|พลาด|ok|false', regex=True), 'log_id']
    list_log_id = list_log_id.tolist()

    return list_log_id

In [ ]:
def calculate_target(df):
    list_unblock_log = get_log_unblock(df)

    df['next_action'] = df.groupby('user_id')['action'].shift(-1)
    df['is_block'] = df['next_action'].isin(['block', 'reblock']) & ~df['log_id'].isin(list_unblock_log)

    n_blocks = df.groupby('user_id').rolling('2d', min_periods=1, on='log_timestamp')['is_block'].sum()
    n_blocks = n_blocks.astype(int)
    n_blocks = n_blocks.reset_index(level=0, drop=True)
    n_blocks.reset_index(drop=True, inplace=True)

    df['n_blocks'] = n_blocks

    df['is_block_next_period'] = df['n_blocks'].ge(3).astype(int)

    df.drop(columns=['next_action', 'is_block', 'n_blocks'], inplace=True)

# Import Data

In [8]:
path = Path(os.getcwd())

project_dir = path.parent
raw_data_dir = project_dir / 'data' / 'raw'

In [9]:
# user log
tree_page_logging = ET.parse(raw_data_dir / 'thwiki-20240920-pages-logging.xml')
root_page_logging = tree_page_logging.getroot()

# language proficiency
with open(raw_data_dir / 'thwiki-20240920-babel.sql', 'r') as f:
    sql_lang_prof = f.read()

In [10]:
df_user_log = parse_data(root=root_page_logging)
df_lang_prof = parse_query(sql_lang_prof)

progress: 10% .
progress: 20% ..
progress: 30% ...
progress: 40% ....
progress: 50% .....
progress: 60% ......
progress: 70% .......
progress: 80% ........
progress: 90% .........


C:\Users\11413929\AppData\Local\Temp\ipykernel_14516\2690832593.py:41: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json.dumps(list_records), orient='records')


In [11]:
df_lang_prof.head(2)

,babel_user,babel_lang,babel_level
0,138412,ab,3
1,91937,af,1


In [12]:
df_user_log.head(2)

,log_id,log_timestamp,user_name,user_id,user_comment,action_type,action,log_title
0,1,2004-12-23T07:51:14Z,Dr.Akarat,174.0,ชาวไอนุ บนเกาะฮอกไกโด ประเทศญี่ปุ่น เมื่อ ค.ศ....,upload,upload,ไฟล์:AinuGroup.jpg
1,2,2004-12-23T09:59:44Z,Dr.Akarat,174.0,เมืองฮาโกดาเตะยามค่ำคืน,upload,upload,ไฟล์:Hakodate.jpg


In [15]:
df_user_block = df_user_log.loc[df_user_log['action'].isin(['block', 'reblock']), ['log_id', 'user_id', 'log_timestamp']]
df_user_unblock = df_user_log.loc[df_user_log['action'].eq('unblock'), ['user_id', 'log_timestamp', 'user_comment']]

df_user_block['log_timestamp'] = pd.to_datetime(df_user_block['log_timestamp'].str.replace('T', ' ').str.replace('Z', ''))
df_user_unblock['log_timestamp'] = pd.to_datetime(df_user_unblock['log_timestamp'].str.replace('T', ' ').str.replace('Z', ''))

df_user_block.sort_values(by='log_timestamp', inplace=True)
df_user_unblock.sort_values(by='log_timestamp', inplace=True)

df_comment = pd.merge_asof(df_user_block, df_user_unblock, by='user_id', on='log_timestamp', tolerance=pd.Timedelta('2d'), direction='forward')

df_comment['user_comment'].value_counts().head(25)

user_comment
เปลี่ยนชื่อผู้ใช้แล้ว                          29
false positive                                 27
แก้ไขเจตนาดี                                   16
OK                                             16
มีช่วงการบล็อกที่ครอบคลุมกว่า                  13
ไม่ได้ก่อกวน                                   11
เปลี่ยนชื่อแล้ว                                10
ปรับช่วงการบล็อกใหม่                           10
แจ้งเปลี่ยนชื่อ                                 8
การแก้ถูกต้องแล้ว                               8
ผิดพลาด                                         7
per [[WP:AN#ข้อความฝากถามแอดมินวิกิพีเดีย]]     7
บล็อกผิดไอพี                                    7
ต้องการระบุเหตุผลในการบล็อกให้ชัดเจน            6
ยกเลิกการบล็อก                                  6
รอตรวจสอบก่อน                                   6
ไม่มีความจำเป็นต้องบล็อกไอพีนาน                 6
อุทธรณ์การบล็อก                                 5
แก้ไขชื่อแล้ว                                   4
ให้เวลาทบทวนตนเองนานมากแล้ว          

There are some keywords which indicates misunderstandings, such as, <code>'false positive'</code>, <code>'ผิด'</code>, <code>'พลาด'</code>, <code>'ok'</code>

# Pre-process

In [14]:
process_log(df_user_log)

calculate_target(df_user_log)

In [44]:
df_user_log

,log_id,log_timestamp,user_id,user_comment,action_type,action,is_block_next_period
0,2544,2005-08-29 21:55:27,1,testing bug?,block,block,0
1,2545,2005-08-29 22:22:37,1,testing bug,block,block,0
2,161699,2008-12-28 18:02:19,1,Cut file size from 30k to 16k (converted to gr...,upload,overwrite,0
3,50,2005-01-10 13:09:40,2,None,delete,delete,0
4,51,2005-01-10 13:11:02,2,เนื้อหาเดิม: 'สวัสดีครับ',delete,delete,0
...,...,...,...,...,...,...,...
1593971,1723590,2024-09-23 11:51:03,488702,None,newusers,autocreate,0
1593972,1723592,2024-09-23 11:53:03,488703,None,newusers,autocreate,0
1593973,1723595,2024-09-23 12:08:26,488704,None,newusers,autocreate,0
1593974,1723613,2024-09-23 12:40:41,488705,None,newusers,autocreate,0


In [39]:
df_user_log['is_block_next_period'].sum()

np.int64(65003)

In [40]:
df_user_log['n_blocks'].gt(0).sum()

np.int64(152201)

In [20]:
df_user_log.reset_index(drop=True, inplace=True)

In [23]:
df_user_log

,log_id,log_timestamp,user_id,user_comment,action_type,action,next_action,is_block
0,2544,2005-08-29 21:55:27,1,testing bug?,block,block,block,True
1,2545,2005-08-29 22:22:37,1,testing bug,block,block,overwrite,False
2,161699,2008-12-28 18:02:19,1,Cut file size from 30k to 16k (converted to gr...,upload,overwrite,NaN,False
3,50,2005-01-10 13:09:40,2,None,delete,delete,delete,False
4,51,2005-01-10 13:11:02,2,เนื้อหาเดิม: 'สวัสดีครับ',delete,delete,delete,False
...,...,...,...,...,...,...,...,...
1593971,1723590,2024-09-23 11:51:03,488702,None,newusers,autocreate,NaN,False
1593972,1723592,2024-09-23 11:53:03,488703,None,newusers,autocreate,NaN,False
1593973,1723595,2024-09-23 12:08:26,488704,None,newusers,autocreate,NaN,False
1593974,1723613,2024-09-23 12:40:41,488705,None,newusers,autocreate,NaN,False


In [77]:
df_user_log.n_blocks.isnull().sum()

np.int64(0)

In [78]:
df_user_log['n_blocks'].gt(0).astype(int)

2543       1
2544       1
157222     0
49         0
50         0
          ..
1604742    0
1604744    0
1604747    0
1604762    0
1604768    0
Name: n_blocks, Length: 3271944, dtype: int64

155        87720
514       136273
516       136320
695       159759
857       179364
          ...   
10410    1158487
10414    1159015
13680    1629856
13681    1629858
13684    1630763
Name: log_id, Length: 83, dtype: int64

In [124]:
df_prep.shape

(1604779, 5)

In [126]:
df1 = pd.DataFrame({ 'datetime':['2020-01-01 07:00:00', '2020-01-01 08:00:00', '2020-01-03 07:00:00', '2020-01-03 07:02:00', '2020-01-03 07:02:00', '2020-01-04 07:02:00', '2020-01-05 07:02:00'],
                     'key': [1, 1, 1, 1, 2, 2, 2],
                     'sample':  ['create', 'create', 'delete', 'delete', 'block', 'unblock', 'create'] })


In [133]:
df1

,datetime,key,sample
0,2020-01-01 07:00:00,1,create
1,2020-01-01 08:00:00,1,create
2,2020-01-03 07:00:00,1,delete
3,2020-01-03 07:02:00,1,delete
4,2020-01-03 07:02:00,2,block
5,2020-01-04 07:02:00,2,unblock
6,2020-01-05 07:02:00,2,create


In [128]:
df1['datetime'] = pd.to_datetime(df1['datetime'])

In [135]:
pd.merge_asof(df1, df1, left_index=False, right_index=False, by='key', on='datetime', tolerance=pd.Timedelta(days=1))

,datetime,key,sample_x,sample_y
0,2020-01-01 07:00:00,1,create,create
1,2020-01-01 08:00:00,1,create,create
2,2020-01-03 07:00:00,1,delete,delete
3,2020-01-03 07:02:00,1,delete,delete
4,2020-01-03 07:02:00,2,block,block
5,2020-01-04 07:02:00,2,unblock,unblock
6,2020-01-05 07:02:00,2,create,create


In [117]:
pd.merge_asof( df2, df1, on='datetime', tolerance=pd.Timedelta('1s') )

,datetime,sample_x,sample_y
0,2017-01-01 00:00:00.300,0,100
1,2017-01-01 00:00:00.600,1,100
2,2017-01-01 00:00:00.900,2,100
3,2017-01-01 00:00:01.200,3,101
